In [1]:
from transformers import pipeline
import librosa
import sounddevice as sd

In [2]:
model_pth = r"D:\model_code\server_models\wav2vec2\trial_2\adele\wav2vec2-finetune"

In [3]:
pipe = pipeline("audio-classification", model=model_pth)

In [4]:
def get_label(x):
    y = None
    mx_n = -1
    for i in x:
        if i['score'] > mx_n:
            mx_n = i['score']
            y = i['label']
    return y, mx_n
    return y if mx_n > 0.99 and y != 'unknown' else None


Inference

In [5]:
sr = 16000
fs = 44100
buffer_duration = 2 # seconds
buffer_size = buffer_duration * fs

In [6]:
while True:
    try:
        # print("Recording...")
        # Record audio for 2 seconds
        buffer = sd.rec(buffer_size, channels=1, dtype='float32')
        sd.wait()  # Wait until recording is finished
        sd.play(buffer,fs)
        sd.wait()  # Wait until recording is finished
        # Process the recorded audio
        y = buffer.squeeze()[fs:]
        data = librosa.resample(y=y,orig_sr=fs,target_sr=sr)
        l = get_label(pipe(data))
        print(l or '',end=' ')
        # break
        # clear_output(wait=True)
    except KeyboardInterrupt:
        print("Stream stopped by user")
        break

('unknown', 0.9995005130767822) ('unknown', 0.9994996786117554) ('unknown', 0.9994950294494629) Stream stopped by user


In [12]:
import librosa
import glob
pth = r'D:\keyword_spotting\web_app\static\audios\2024-07-29_19-19'
files = [j for i in glob.glob(f'{pth}/*') for j in glob.glob(f'{i}/*')]
for f in files:
    y, sr = librosa.load(f, sr=None)
    l = get_label(pipe(y))
    if not l[0] == 'unknown':
        print(f.replace('\\','/').split('/')[-1],l)

0_adele.wav ('adele', 0.9990003705024719)
10_adele.wav ('adele', 0.9950312376022339)
14_adele.wav ('adele', 0.7823439836502075)
1_adele.wav ('adele', 0.9868614077568054)
9_adele.wav ('adele', 0.992116391658783)
13_hilfe.wav ('adele', 0.9725024700164795)
1_hilfe.wav ('adele', 0.8856857419013977)
20_hilfe.wav ('adele', 0.7850538492202759)
24_hilfe.wav ('adele', 0.9988188147544861)
30_hilfe.wav ('adele', 0.9916595816612244)
36_hilfe.wav ('adele', 0.9934617280960083)
38_hilfe.wav ('adele', 0.7079593539237976)
40_hilfe.wav ('adele', 0.6062816977500916)
47_hilfe.wav ('adele', 0.9821916222572327)
7_hilfe.wav ('adele', 0.9512498378753662)
8_hilfe.wav ('adele', 0.9990257024765015)
25_unknown.wav ('adele', 0.9958192110061646)
312_unknown.wav ('adele', 0.9873260259628296)
31_unknown.wav ('adele', 0.9736422896385193)
398_unknown.wav ('adele', 0.9633371829986572)
459_unknown.wav ('adele', 0.9533089995384216)
500_unknown.wav ('adele', 0.9503385424613953)
512_unknown.wav ('adele', 0.9753772020339966)

In [ ]:
from audiostream import AudioStream, AudioInput
import numpy as np
from collections import deque
import time

# Define your function that processes 1-second audio chunks
def process_audio_chunk(audio_chunk):
    # Your processing code here
    print("Processing audio chunk of length:", len(audio_chunk))

# Audio stream configuration
sample_rate = 16000  # 16 kHz
chunk_duration = 1.0  # 1 second
overlap_duration = 0.5  # 0.5 second

# Calculate the number of samples per chunk and overlap
chunk_samples = int(chunk_duration * sample_rate)
overlap_samples = int(overlap_duration * sample_rate)

# Buffer to store audio data
buffer = deque(maxlen=chunk_samples + overlap_samples)

# Callback to process audio data
def audio_callback(input_stream):
    audio_data = np.frombuffer(input_stream.read(), dtype=np.float32)
    
    # Append new audio data to buffer
    buffer.extend(audio_data)
    
    # Process the buffer if it has enough samples for a chunk
    while len(buffer) >= chunk_samples:
        # Extract chunk and pass it to the processing function
        audio_chunk = np.array(buffer)[:chunk_samples]
        process_audio_chunk(audio_chunk)
        
        # Remove the processed chunk while keeping the overlap part in the buffer
        for _ in range(chunk_samples - overlap_samples):
            buffer.popleft()

# Set up the audio input stream
stream = AudioInput(callback=audio_callback, channels=1, samplerate=sample_rate, buffersize=chunk_samples)

# Start recording
stream.start()

print("Recording audio. Press Ctrl+C to stop.")
try:
    while True:
        time.sleep(1)  # Keep the script running
except KeyboardInterrupt:
    print("Stopped recording.")
    stream.stop()


ModuleNotFoundError: No module named 'audiostream'